In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn.metrics as sk_metrics
import tempfile
import os

# Preset matplotlib figure sizes.
matplotlib.rcParams['figure.figsize'] = [9, 6]

print(tf.__version__)
# To make the results reproducible, set the random seed value.
random_seed = 22
tf.random.set_seed(random_seed)

2024-11-30 11:44:26.571037: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-30 11:44:26.573014: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-30 11:44:26.604127: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-30 11:44:26.604164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-30 11:44:26.605030: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

2.15.0


In [2]:
test_data_path = 'test_data_ver1.csv'
train_and_validation_data_path = 'train_data_ver3.csv'

dataset = pd.read_csv(train_and_validation_data_path)
dataset = dataset[['home_team_win', 'home_batting_batting_avg_10RA', 'home_batting_onbase_perc_10RA', 'home_batting_onbase_plus_slugging_10RA', 'home_batting_leverage_index_avg_10RA', 'home_batting_RBI_10RA', 'away_batting_batting_avg_10RA', 'away_batting_onbase_perc_10RA', 'away_batting_onbase_plus_slugging_10RA', 'away_batting_leverage_index_avg_10RA', 'away_batting_RBI_10RA', 'home_pitching_earned_run_avg_10RA', 'home_pitching_SO_batters_faced_10RA', 'home_pitching_H_batters_faced_10RA', 'home_pitching_BB_batters_faced_10RA', 'away_pitching_earned_run_avg_10RA', 'away_pitching_SO_batters_faced_10RA', 'away_pitching_H_batters_faced_10RA', 'away_pitching_BB_batters_faced_10RA', 'home_pitcher_earned_run_avg_10RA', 'home_pitcher_SO_batters_faced_10RA', 'home_pitcher_H_batters_faced_10RA', 'home_pitcher_BB_batters_faced_10RA', 'away_pitcher_earned_run_avg_10RA', 'away_pitcher_SO_batters_faced_10RA', 'away_pitcher_H_batters_faced_10RA', 'away_pitcher_BB_batters_faced_10RA', 'home_batting_leverage_index_avg_mean', 'home_batting_leverage_index_avg_std', 'home_batting_leverage_index_avg_skew', 'home_batting_wpa_bat_mean', 'home_batting_wpa_bat_std', 'home_batting_wpa_bat_skew', 'away_batting_leverage_index_avg_mean', 'away_batting_leverage_index_avg_std', 'away_batting_leverage_index_avg_skew', 'away_batting_wpa_bat_mean', 'away_batting_wpa_bat_std', 'away_batting_wpa_bat_skew', 'home_pitching_earned_run_avg_mean', 'home_pitching_earned_run_avg_std', 'home_pitching_earned_run_avg_skew', 'home_pitching_SO_batters_faced_mean', 'home_pitching_SO_batters_faced_std', 'home_pitching_SO_batters_faced_skew', 'home_pitching_H_batters_faced_mean', 'home_pitching_H_batters_faced_std', 'home_pitching_H_batters_faced_skew', 'home_pitching_BB_batters_faced_mean', 'home_pitching_BB_batters_faced_std', 'home_pitching_BB_batters_faced_skew', 'home_pitching_leverage_index_avg_mean', 'home_pitching_leverage_index_avg_std', 'home_pitching_leverage_index_avg_skew', 'home_pitching_wpa_def_mean', 'home_pitching_wpa_def_std', 'home_pitching_wpa_def_skew', 'away_pitching_earned_run_avg_mean', 'away_pitching_earned_run_avg_std', 'away_pitching_earned_run_avg_skew', 'away_pitching_SO_batters_faced_mean', 'away_pitching_SO_batters_faced_std', 'away_pitching_SO_batters_faced_skew', 'away_pitching_H_batters_faced_mean', 'away_pitching_H_batters_faced_std', 'away_pitching_H_batters_faced_skew', 'away_pitching_BB_batters_faced_mean', 'away_pitching_BB_batters_faced_std', 'away_pitching_BB_batters_faced_skew', 'away_pitching_leverage_index_avg_mean', 'away_pitching_leverage_index_avg_std', 'away_pitching_leverage_index_avg_skew', 'away_pitching_wpa_def_mean', 'away_pitching_wpa_def_std', 'away_pitching_wpa_def_skew', 'home_pitcher_earned_run_avg_mean', 'home_pitcher_earned_run_avg_std', 'home_pitcher_earned_run_avg_skew', 'home_pitcher_leverage_index_avg_mean', 'home_pitcher_leverage_index_avg_std', 'home_pitcher_leverage_index_avg_skew', 'home_pitcher_wpa_def_mean', 'home_pitcher_wpa_def_std', 'home_pitcher_wpa_def_skew', 'away_pitcher_earned_run_avg_mean', 'away_pitcher_earned_run_avg_std', 'away_pitcher_earned_run_avg_skew', 'away_pitcher_leverage_index_avg_mean', 'away_pitcher_leverage_index_avg_std', 'away_pitcher_leverage_index_avg_skew', 'away_pitcher_wpa_def_mean', 'away_pitcher_wpa_def_std', 'away_pitcher_wpa_def_skew']]

In [3]:
class LogisticRegressionModel(tf.keras.Model):
    def __init__(self):
        super(LogisticRegressionModel, self).__init__()
        self.dense = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        return self.dense(inputs)
  

In [4]:
def cross_validation(num_chunks: int):
    chunk_size = len(dataset) // num_chunks
    chunks = [dataset.iloc[i * chunk_size:(i + 1) * chunk_size] for i in range(num_chunks)]
    # Handle any remaining rows (if the dataset isn't perfectly divisible)
    if len(dataset) % num_chunks != 0:
        chunks[-1] = pd.concat([chunks[-1], dataset.iloc[num_chunks * chunk_size:]])

    best_accuracy, best_loss, best_model = -1, -1, -1

    for v in range(num_chunks):
        validation_dataset = chunks[v]
        train_dataset = dataset.drop(chunks[v].index)

        # Split the features and the target values
        x_train = train_dataset.drop('home_team_win', axis=1)
        y_train = train_dataset['home_team_win']
        x_validation = validation_dataset.drop('home_team_win', axis=1)
        y_validation = validation_dataset['home_team_win']

        # Convert the features and the target values to TensorFlow tensors
        x_train = tf.constant(x_train, dtype=tf.float64)
        y_train = tf.constant(y_train, dtype=tf.float64)
        x_validation = tf.constant(x_validation, dtype=tf.float64)
        y_validation = tf.constant(y_validation, dtype=tf.float64)

        # Instantiate the model
        model = LogisticRegressionModel()

        # Compile the model
        model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
                    loss=tf.keras.losses.BinaryCrossentropy(),
                    metrics=['accuracy'])
        
        model.fit(x_train, y_train, epochs=5, batch_size=1)
        loss, accuracy = model.evaluate(x_validation, y_validation)
        print(f"Loss: {loss}, Accuracy: {accuracy}")

        if best_accuracy == -1 or accuracy > best_accuracy:
            best_accuracy = accuracy
            best_loss = loss
            best_model = model

    return best_accuracy, best_loss, best_model



In [5]:
best_accuracy, best_loss, best_model = cross_validation(5)
print(f"Best accuracy: {best_accuracy}, Best loss: {best_loss}")

Epoch 1/5
8854/8854 [==============================] - 7s 820us/step - loss: 0.7769 - accuracy: 0.5217
Epoch 2/5
8854/8854 [==============================] - 7s 845us/step - loss: 0.7663 - accuracy: 0.5381
Epoch 3/5
8854/8854 [==============================] - 8s 872us/step - loss: 0.7678 - accuracy: 0.5311
Epoch 4/5
8854/8854 [==============================] - 7s 835us/step - loss: 0.7659 - accuracy: 0.5313
Epoch 5/5
70/70 [==============================] - 0s 1ms/step - loss: 0.7789 - accuracy: 0.5138
Loss: 0.7789482474327087, Accuracy: 0.5137822031974792
Epoch 1/5
8854/8854 [==============================] - 8s 868us/step - loss: 0.7697 - accuracy: 0.5273
Epoch 2/5
8854/8854 [==============================] - 7s 838us/step - loss: 0.7661 - accuracy: 0.5295
Epoch 3/5
8854/8854 [==============================] - 7s 810us/step - loss: 0.7742 - accuracy: 0.5204
Epoch 4/5
8854/8854 [==============================] - 7s 833us/step - loss: 0.7663 - accuracy: 0.5309
Epoch 5/5
70/70 [=======